In [ ]:
"""This script generates the features for classification in subtask B. Set the TRAIN flag to True to 
    generate features for the training data, and to False to generate features for test data"""

import json, re
import numpy as np 
import pandas as pd

TRAIN = False

In [ ]:
# reading data
from load import parse_dataset

In [ ]:
if TRAIN:
    dataset='../datasets/train/SemEval2018-T3-train-taskB_emoji.txt'
    corpus, _ = parse_dataset(dataset)
    corpus_preprocessed = json.load(open('../extra_resources/train_preprocessed.txt','r'))
else:
    dataset='../datasets/goldtest_TaskB/SemEval2018-T3_gold_test_taskB_emoji.txt'
    corpus, _ = parse_dataset(dataset)
    corpus_preprocessed = json.load(open('../extra_resources/test_preprocessed.txt','r'))

In [ ]:
if TRAIN:
    corpus_preprocessed = json.load(open('../extra_resources/train_preprocessed.txt','r'))
else:
    corpus_preprocessed = json.load(open('../extra_resources/test_preprocessed.txt','r'))

In [ ]:
def chunkIt(seq, n):
    """splits the list into n approximately equal sub-lists. source: goo.gl/VrHKeR"""
    avg = len(seq) / float(n)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

In [ ]:
# intensity features - 3 binarized features for splitted tweets which show: 
# 1) the intensity of the left half
# 2) the intensity of the right half
# 3) the difference between the polarities of left and right halves 

# to run this, first download stanford corenlp
# install pycorenlp: pip install pycorenlp
# then enter this on terminal, within the diretcory of corenlp 
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -annotators "sentiment" -port 9000 -timeout 30000

from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

feats_1 = []

for text in corpus:
    part1, part2 = chunkIt(text, 2)
    output1 = int(nlp.annotate(part1, properties={
                           'annotators': 'sentiment',
                             'outputFormat': 'json'
                           })['sentences'][0]['sentimentValue'])
    output2 = int(nlp.annotate(part2, properties={
                           'annotators': 'sentiment',
                             'outputFormat': 'json'
                           })['sentences'][0]['sentimentValue'])    
    leftIntensity = rightIntensity = polarityDiff = 0
    if output1 in [0,4]:
        leftIntensity = 1
    if output2 in [0,4]:
        rightIntensity = 1
    if (output1>2 and output2<2) or (output1<2 and output2>2):
        polarityDiff = 1
    feats_1.append(np.array([leftIntensity, rightIntensity, polarityDiff]))

In [ ]:
# contrast 
df = pd.read_csv('../extra_resources/Emoji_Sentiment_Data_v1.0.csv')

df = df[['Emoji', 'Negative', 'Neutral', 'Positive']]
tuples = [tuple(x) for x in df.values]
# tuples are of the form (emoji, negative, neutral, positive)

idx2lb = {0:-1, 1:0, 2:1}
emoji_sentimens = {}
for val in tuples:
    emoji_sentimens[val[0]] = idx2lb[np.argmax(np.array(val[1:]))]

def extractEmoticon(tweet):
    """returns all the emoticons in tweet"""
    return re.findall(r'[\U0001f600-\U0001f650]', ' '.join(tweet))

twts = [extractEmoticon(twt[0]) for twt in corpus_preprocessed]
twts = [[emoji_sentimens[emoji] for emoji in twt] for twt in twts]

def extractHashtag(tweet):
    t = tweet.split(' ')
    text = []
    hashtagText = []
    oneHashtag = []
    flag = 0
    for w in t:
        if w == "<hashtag>":
            flag = 1
            continue
        if flag == 1:
            if w == "</hashtag>":
                hashtagText.append(oneHashtag)
                oneHashtag = []
                flag = 0
            else:
                oneHashtag.append(w)
        else:
            text.append(w)
    return text, hashtagText

txt = [extractHashtag(tweet) for tweet in corpus_preprocessed]

assert len(txt) == len(twts)
txt = [(txt[i][0], txt[i][1], twts[i]) for i in range(len(twts))]

# 0: very negative
# 1: negative 
# 2: neutral 
# 3: positive 
# 4: very positive 
    
def sentiment(txt):
    """compute sentiment for text"""
    txt = ' '.join(txt)
    if not len(txt): return 2
    output = int(nlp.annotate(txt, properties={
                              'annotators': 'tokenize,ssplit,pos,depparse,parse,sentiment',
                                'outputFormat': 'json'
                              })['sentences'][0]['sentimentValue'])
    return output

def contrast(twt):
    """search for emotion contrast in hastag, emoticon and tweet text"""
    contrast = 0 # contrast flag
    txt_sentiment = sentiment(twt[0])
    htag_sentiment = [sentiment(h) for hash_segments in twt[1] for h in hash_segments]
    emoji_sentiment = twt[2]

    if (txt_sentiment in {2,3,4}) and (set(htag_sentiment) & {0,1}):
        contrast = 1
    elif (txt_sentiment in {0,1}) and (set(htag_sentiment) & {3,4}): # maybe later try with adding 2
        contrast = 1
    elif (txt_sentiment in {2,3,4}) and (set(emoji_sentiment) & {-1}):
        contrast = 1
    elif (txt_sentiment in {0,1}) and (set(emoji_sentiment) & {1}):
        contrast = 1
    elif {-1,1} in set(emoji_sentiment):
        contrast = 1
    elif ({0,4} in set(htag_sentiment)) or ({0,3} in set(htag_sentiment)) or ({1,4} in set(htag_sentiment)):
        contrast = 1
    elif (set(htag_sentiment) & {0,1}) and (set(emoji_sentiment) & {1}):
        contrast = 1
    elif (set(htag_sentiment) & {3,4}) and (set(emoji_sentiment) & {-1}):
        contrast = 1
    return contrast

contrast_feats = [np.array([contrast(twt)]) for twt in txt]

In [ ]:
# ekphrasis-based features (extracted from pre-processed data)  2*24 features

tags =  ['<allcaps>', '<annoyed>', '<censored>', '<date>', '<elongated>', '<emphasis>', '<happy>',
         '<hashtag>', '<heart>', '<kiss>', '<laugh>', '<money>', '<number>', '<percent>', '<phone>',
         '<repeated>', '<sad>', '<shocking>', '<surprise>', '<time>', '<tong>', '<url>', '<user>',
         '<wink>']

In [ ]:
def tweet_vecs(twt, n=2):
    """extract a feature vector for a single tweet, based on the counts of the annotation tags
        split the tweet to n equal parts and computes the same features for each part"""
    twt = twt.split()
    chunks = chunkIt(twt, n)
    
    scores = []
    
    for chunk in chunks:
        for tag in tags:
            scores.append(sum(1 for t in chunk if t == tag))
    return scores
    

def feats(text):
    """apply the tweet_vecs function on all tweets and return a result in a list"""
    return [tweet_vecs(twt) for twt in text]

ekphrasis_feats = [np.array(v) for v in feats(corpus_preprocessed)]

In [ ]:
# polarity features, which didn't result in good perfoemance and hence, were discarded

from ekphrasis.utils.nlp import polarity
polarity_flag = False

polarity_vectors = []
for tweet in corpus_preprocessed:
    chunks = chunkIt(tweet, 2)
    polarity_vectors.append(np.concatenate(((polarity(chunks[0])[1], polarity(chunks[1])[1])), axis=0))

assert len(ekphrasis_feats) == len(polarity_vectors)

if polarity_flag: 
    ekphrasis_feats = [np.concatenate((ekphrasis_feats[i], polarity_vectors[i])) for i in range(len(ekphrasis_feats))]

# np.save('ekphrasis_feats', np.array(ekphrasis_feats))

In [ ]:
# no-chunk (i.e. whole tweet) ekphrasis feats 
def tweet_vecs(twt):
    words = twt.split()    
    scores = []
    
    for tag in tags:
        scores.append(sum(1 for word in words if word == tag))
    return scores
    

def feats(text):
    """apply the tweet_vecs function on all tweets and return a result in a list"""
    return [tweet_vecs(twt) for twt in text]

ekphrasis_feats_no_chunks = [np.array(v) for v in feats(corpus_preprocessed)]
# np.save('ekphrasis_feats_no_chunks', np.array(ekphrasis_feats_no_chunks))

In [ ]:
# Vader feats 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from itertools import groupby
from collections import Counter

vader = SentimentIntensityAnalyzer()

maxlength = max([len(s) for s in corpus_preprocessed])

def senti_patterns(text):
    patterns = []
    """receives list of list of tokens (sentences), returns rise(R) and fall(F) patterns in sentiment scores"""
    for sentence in text:
        sent_pattern = []
        for word in sentence:
            senti = vader.polarity_scores(word)['compound']
            if senti > 0:
                sent_pattern.append('R')
            elif senti < 0:
                sent_pattern.append('F')
        sent_pattern = [x[0] for x in groupby(sent_pattern)]
        if not sent_pattern: sent_pattern.append('S') 
        patterns.append(sent_pattern)
    return patterns

pat = senti_patterns(corpus_preprocessed)

pats_with_bigrams = []
for tags in pat:
    bigrams = list(zip(tags, tags[1:]))
    tags.extend(bigrams)
    pats_with_bigrams.append(tags)

vader_vecs = [] 

# each count vector is a 5-diemnsional numpy array denoting the counts in the following order:
############################# ['S', 'R', 'F', ('R', 'F'), ('F', 'R')] ######################

for pat in pats_with_bigrams:
    counts = Counter(pat)
    vader_vecs.append(np.array([counts['S'],counts['R'],counts['F'],counts[('R', 'F')], counts[('F', 'R')]]))
    
# np.save('test_vader_feats.npy', np.array(vader_vecs))

In [ ]:
# load new Topic Modeling features
if TRAIN:
    topicFeats_2chunks = np.load('./topic_feats_10_2chunks.npy')
    topicFeats_no_chunks = np.load('./topic_feats_10_no_chunks.npy')
else:
    topicFeats_2chunks = np.load('./test_topic_feats_10_2chunks.npy')
    topicFeats_no_chunks = np.load('./test_topic_feats_10_no_chunks.npy')

In [ ]:
# concatenate all the features 
features = np.concatenate((feats_1, 
                           contrast_feats, 
                           ekphrasis_feats, 
                           ekphrasis_feats_no_chunks, 
                           vader_vecs,
                           topicFeats_2chunks,
                           topicFeats_no_chunks), axis=1)

In [ ]:
if TRAIN:
    np.save('all_train_feats_TaskB.npy', features)
else:
    np.save('all_test_feats_TaskB.npy', features)